# Part 3

### Question 3: Parameter-Sharing Scheme for 2D Linear Equivariant Layer

In [ ]:
import torch
import matplotlib.pyplot as plt

M = 8
N = 7

W = torch.zeros(M, N, M, N)
A = torch.randn(M, N)

for i1 in range(M):
    for j1 in range(N):
        for i2 in range(M):
            for j2 in range(N):
                if i1 == i2 and j1 == j2:
                    W[i1, j1, i2, j2] = 1
                elif i1 == i2 and j1 != j2:
                    W[i1, j1, i2, j2] = 2
                elif i1 != i2 and j1 == j2:
                    W[i1, j1, i2, j2] = 3
                else:
                    W[i1, j1, i2, j2] = 4

plt.imshow(W.reshape(M * N, M * N).to(torch.int32), cmap="tab10")

### Question 5: Parameter-Sharing Scheme for 3D Linear Equivariant Layer

In [ ]:
import torch
import matplotlib.pyplot as plt

M = 4
N = 3
K = 5

W = torch.zeros(M, N, K, M, N, K)
A = torch.randn(M, N, K)

for i1 in range(M):
    for j1 in range(N):
        for p1 in range(K):
            for i2 in range(M):
                for j2 in range(N):
                    for p2 in range(K):
                        if i1 == i2 and j1 == j2 and p1 == p2:
                            W[i1, j1, p1, i2, j2, p2] = 1
                        if i1 == i2 and j1 == j2 and p1 != p2:
                            W[i1, j1, p1, i2, j2, p2] = 2
                        if i1 == i2 and j1 != j2 and p1 == p2:
                            W[i1, j1, p1, i2, j2, p2] = 3
                        if i1 == i2 and j1 != j2 and p1 != p2:
                            W[i1, j1, p1, i2, j2, p2] = 4
                        if i1 != i2 and j1 == j2 and p1 == p2:
                            W[i1, j1, p1, i2, j2, p2] = 5
                        if i1 != i2 and j1 == j2 and p1 != p2:
                            W[i1, j1, p1, i2, j2, p2] = 6
                        if i1 != i2 and j1 != j2 and p1 == p2:
                            W[i1, j1, p1, i2, j2, p2] = 7
                        if i1 != i2 and j1 != j2 and p1 != p2:
                            W[i1, j1, p1, i2, j2, p2] = 0

plt.imshow(W.reshape(M * N * K, M * N * K).to(torch.int32), cmap="tab10")

# Part 5

### Question 4: Challenges encountered during Implementation:

##### Numeric Errors:

The first challenge encountered is in the implementation of the invariant and equivariant layers.
The main implementation challenge rose from the fact that in the lecture, the equivariant layer is formulated as follows:

$$ F(x) : \mathbb{R}^{n \times d} \rightarrow \mathbb{R}^{n \times d'} $$

$$ F(x)_j = \sum _{i=1} ^ {d} L_{ij}(x) $$ 
where $L_{ij}(x)$ is a single feature linear equivariant layer.

Technically, this implementation is indeed correct, but the summation over all $L_{ij}(x)$ might causes layer outputs to blow-up.  
As result, the outputs of the $F \circ a \circ F ...$ become very large.

Our network is composed of these layers $\phi \circ F \circ a \circ F ...$, when $\phi$ is the sigmoid function that returns values between 0 and 1.

Since the last layer of the network is a sigmoid function, and the results of the previous layers are very large (their absolute value), the sigmoid function saturates and returns either 0.0 or 1.0. Because the sigmoid function got saturated, the propagated gradients become 0, hence the network does not learn.

To resolve this issue we defined the equivariant layer as follows:

$$ F(x)_j = \frac{1}{d} \sum _{i=1} ^ {d} L_{ij}(x) $$ 

This formulation still retains the equivariance property, but it prevents the layer outputs from blowing-up.

*Note: We applied the same averaging technique to the invariant layers as well.*

##### Overfitting:

Another big issue we encountered was overfitting. To overcome it, we added an option to dynamically generate the data every time the `Dataset` is accessed. 
This way, the model never sees the same data twice, and not able to overfit. That indeed resolved completely the overfitting issue.
For the comparative analysis, we didn't use this option.

##### Symmetrization Network:

The symmetrization network is a very powerful tool to learn equivariant functions. However, it is computationally expensive
and tricky to implement efficiently. Our implementation balances performance and memory utilization by forwarding through the network multiple 
permuted versions of the input data at once (by creating a super-batch). We can control that number of forwarded permutations to balance performance and memory (more permutations - better performance but higher memory utilization).

### Question 6 and 7:

The reason that the equivariant architecture is especially suited for this task is the following:

Given $d$ dimensional random vector $x = (x_1, x_2, ..., x_d) \sim \mathcal{D^d}$, the formula for the empirical variance is as follows:

$$ {Var}_{em}(x) = \frac{1}{d - 1} \sum_{i=1}^{d} (x_i - \bar{x})^2 $$

$$ \bar{x} = \frac{1}{d} \sum_{i=1}^{d} x_i $$

Notice that this calculation can be easily achieved by the the following invariant architecture:

($I_{d \times d}$ denotes the identity matrix, $1_{d \times d}$ denotes the matrix filled with ones, and $0_{d}$ denotes the zero vector)

Let $N = \phi \circ \alpha \circ F$ be an invariant network, where $\phi$ is invariant layer, $\alpha$ is pointwise activation, and $F$ is an equivariant layer.

* Define $F : \mathbb{R}^d \rightarrow \mathbb{R}^d$ as follows: $ F = 1 \cdot I_{d \times d} + \frac{-1}{d} \cdot {1}_{d \times d} + 0_{d}$
(recall that general equivariant $F$ is of the form $ F = \alpha \cdot I_{d \times d} + \beta \cdot {1}_{d \times d} + b$)

* Define $\alpha : \mathbb{R} \rightarrow \mathbb{R}$ as follows: $ \alpha(x) = x^2 $

* Define $\phi : \mathbb{R}^d \rightarrow \mathbb{R}$ as follows: $ \phi = \frac{1}{d-1} \cdot 1_{d} + 0$ (recall that general invariant $\phi$ is of the form $ \phi = \alpha \cdot 1_{d} + b$)

As result, we get:

$$ F(x)_i = x_i - \frac{1}{d} \sum_{j=1}^{d} x_j + 0 = x_i - \bar{x} $$

$$ \alpha(F(x))_i = F(x)_i^2 = (x_i - \bar{x})^2 $$

$$ N(x) = \frac{1}{d-1} \sum_{i=1}^{d} \alpha(F(x))_i = \frac{1}{d-1} \sum_{i=1}^{d} (x_i - \bar{x})^2 = \text{Var}_{em}(x) $$

We showed that using the invariant and equivariant layers only, we were able to calculate the empirical variance of an input sample. 
Since the task of the model is to differentiate between inputs generated from distributions with different variances, the equivariant architecture is especially suited for this task.
Notice the small number of parameters required to calculate the variance (3 parameters in layer $F$ and two parameters in layer $\phi$), hence the optimization process of this architecture is easier than for other architectures.

*Note: We showed how to calculate the variance across a single feature dimension, but the same idea holds for calculating the variance for each feature for element of size $(n \times d)$, and afterwards averaging the empirical variances across the feature dimension to get the final variance of the sample.*

### Question 8:

Currently, we're using the symmetry group $S_n$ over the channel dimensions.
A better symmetry group to use would be $S_n \times S_d$ when $S_n$ acts on the channel dimension and $S_d$ acts on the feature dimension. The reason this symmetry group is suitable is because each feature is a vector of length $d$ generated from a normal distribution, and any permutation of the vector does not change the probability of it being generated, nor the underlying distribution that generated it. Since the model tries to detect the underlying distribution, it should be invariant to permutations of the feature dimensions.

Formally:

$$ \Pr(x_1, x_2, ... x_n \sim \mathcal{N}(0, I) \; | \; x_1, x_2, ... x_n) = 
\Pr(\sigma \cdot x_1, \sigma \cdot x_2, ... \sigma \cdot x_n \sim \mathcal{N}(0, I) \; | \; x_1, x_2, ... ,x_n, \forall \sigma \in S_d) $$

when $x_i$ is a feature vector of length $d$ and $\sigma$ is a permutation of the feature dimensions
(remember that each input sample is composed of $n$ feature vectors of length $d$).